# «Python для сбора данных» — 27.05.2021
*Автор: Анастасия Паршина, НИУ ВШЭ* 

E-mail: a.a.parshina@ya.ru

## Работа с `requests` и `BeautifulSoup`, сбор данных с помощью регулярных выражений. 

Работу с `requests` и `BeautifulSoup` вы также уже разбирали ранее. Сегодня мы попробуем добавить к ним использование регулярных выражений. 

Импортируем необходимые библиотеки:

In [1]:
import re
import requests
from bs4 import BeautifulSoup 
from time import sleep
import pandas as pd

Попробуем изучить результаты выборов в Государственную Думу с сайта российского ЦИКа (например, по Чертановскому округу Москвы). 

In [2]:
url = 'http://www.moscow_city.vybory.izbirkom.ru/region/region/moscow_city?action=show&root=1000259&tvd=100100067796128&vrn=100100067795849&region=77&global=&sub_region=77&prver=0&pronetvd=null&vibid=100100067796128&type=233'

In [3]:
page = requests.get(url)
page.status_code # 200 - все хорошо

200

In [4]:
soup = BeautifulSoup(page.text)
#soup # я закомментировала часть выдач, чтобы файл html по занятию вышел не очень длинным

Попробуем для начала просто вытащить названия районов, которые есть в нашей таблице. Чтобы найти все вхождения тега, который нас интересует, можно воспользоваться методом `find_all()`.

In [5]:
soup.find_all('a') 

[<a href="http://www.moscow_city.vybory.izbirkom.ru/region/izbirkom" id="calendar-btn-mobile">Календарь
 				выборов</a>,
 <a class="rep-name"></a>,
 <a class="nav-link" data-toggle="tab" href="#election-info" id="election-info-name-mobile"></a>,
 <a class="nav-link" data-toggle="tab" href="#standard-reports" id="standard-reports-name-mobile"></a>,
 <a class="nav-link" data-toggle="tab" href="#election-process" id="election-process-name-mobile"></a>,
 <a class="nav-link" data-toggle="tab" href="#digital-election-sites" id="digital-election-sites-name-mobile"></a>,
 <a class="nav-link" data-toggle="tab" href="#digital-election-vote" id="digital-election-vote-name-mobile"></a>,
 <a class="nav-link" data-toggle="tab" href="#digital-remote-election-data" id="digital-remote-election-data-name-mobile"></a>,
 <a class="nav-link" data-toggle="tab" href="#information-about-candidates" id="information-about-candidates-name-mobile"></a>,
 <a class="nav-link" data-toggle="tab" href="#payment-resul

Отлично, но получилось много ненужного. Используем регулярные выражения и вытащим из нашего большого списка только те элементы, в элементах `href` которых содержится слово `region`:

In [7]:
soup.find_all('a', {'href' : re.compile('region')})

[<a href="http://www.moscow_city.vybory.izbirkom.ru/region/izbirkom" id="calendar-btn-mobile">Календарь
 				выборов</a>,
 <a href="http://www.moscow_city.vybory.izbirkom.ru/region/izbirkom" id="calendar-btn">Календарь
 						выборов</a>,
 <a href="region/moscow_city?action=show&amp;root_a=1000259&amp;vrn=100100067795849&amp;region=77&amp;global=&amp;type=0&amp;sub_region=77&amp;prver=0&amp;pronetvd=null">ЦИК России</a>,
 <a href="region/moscow_city?action=show&amp;root_a=1000259&amp;vrn=100100067795849&amp;region=77&amp;global=&amp;type=0&amp;sub_region=77&amp;root=1000259&amp;prver=0&amp;pronetvd=null&amp;tvd=100100067796113">город Москва</a>,
 <a href="region/moscow_city?action=show&amp;root_a=1000259&amp;vrn=100100067795849&amp;region=77&amp;global=&amp;type=0&amp;sub_region=77&amp;root=1000274&amp;prver=0&amp;pronetvd=null&amp;tvd=100100067796128">ОИК №210</a>,
 <a href="http://www.moscow_city.vybory.izbirkom.ru/region/region/moscow_city?action=show&amp;root=1000259&amp;tvd=100100

Уже лучше. Посмотрим на один из элементов: 

In [8]:
soup.find_all('a', {'href' : re.compile('region')})[-1]

<a href="http://www.moscow_city.vybory.izbirkom.ru/region/region/moscow_city?action=show&amp;tvd=100100067796128&amp;vrn=100100067795849&amp;region=77&amp;global=&amp;sub_region=77&amp;prver=0&amp;pronetvd=null&amp;vibid=27720001659801&amp;type=233" style="text-decoration: none">район Чертаново Южное</a>

И обратимся к тексту, который содержится в тегах: 

In [14]:
soup.find_all('a', {'href' : re.compile('region')})[-1].text

'район Чертаново Южное'

Значит, можем сохранять только те значения, которые содержат слово `район`. Сохраним их в словарь `districts`, где ключом будет название р-на, а значением — еще один словарь (!), где ключом будет номер УИКа, а значением — список (данные из соответствующего столбца таблицы). Но сначала посмотрим, как это будет работать на примере одного р-на. 

In [15]:
url0 = soup.find_all('a', {'href' : re.compile('region')})[-1].get('href') # забираем ссылку
url0

'http://www.moscow_city.vybory.izbirkom.ru/region/region/moscow_city?action=show&tvd=100100067796128&vrn=100100067795849&region=77&global=&sub_region=77&prver=0&pronetvd=null&vibid=27720001659801&type=233'

In [11]:
# это страница с данными по УИКам выбранного нами района
page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text)
#soup0

In [12]:
soup0.find_all('td', {'class':re.compile('text-left')})[-1].text

'14. Политическая партия СПРАВЕДЛИВАЯ РОССИЯ'

In [16]:
re.findall(r'[А-Яа-яЁё]\D+', soup0.find_all('td', {'class':re.compile('text-left')})[-1].text)

['Политическая партия СПРАВЕДЛИВАЯ РОССИЯ']

In [17]:
# названия строк заключено в тегах td, можно уточнить class = re.compile('text-left')
rows = []
for i in soup0.find_all('td', attrs={'class':re.compile('text-left')}):
    rows.append(re.findall(r'[А-Яа-яЁё]\D+', i.text)[0])
    
rows

['Число избирателей, внесенных в список избирателей на момент окончания голосования',
 'Число избирательных бюллетеней, полученных участковой избирательной комиссией',
 'Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно',
 'Число избирательных бюллетеней, выданных в помещении для голосования в день голосования',
 'Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования ',
 'Число погашенных избирательных бюллетеней',
 'Число избирательных бюллетеней, содержащихся в переносных ящиках для голосования ',
 'Число избирательных бюллетеней, содержащихся в стационарных ящиках для голосования',
 'Число недействительных избирательных бюллетеней',
 'Число действительных избирательных бюллетеней',
 'Число открепительных удостоверений, полученных участковой избирательной комиссией',
 'Число открепительных удостоверений, выданных на избирательном участке до дня голосования',
 'Число избирателей, проголосовавших по открепительным удостов

In [18]:
# названия УИКов заключено в тегах th, можно уточнить class="text-center"

UIK = soup0.find_all('th', attrs={'class':'text-center'})
UIK

[<th class="fix-col third-fix-col text-center"><nobr><b>Сумма</b></nobr></th>,
 <th class="text-center"><nobr>УИК №2078</nobr></th>,
 <th class="text-center"><nobr>УИК №2079</nobr></th>,
 <th class="text-center"><nobr>УИК №2080</nobr></th>,
 <th class="text-center"><nobr>УИК №2081</nobr></th>,
 <th class="text-center"><nobr>УИК №2082</nobr></th>,
 <th class="text-center"><nobr>УИК №2083</nobr></th>,
 <th class="text-center"><nobr>УИК №2084</nobr></th>,
 <th class="text-center"><nobr>УИК №2085</nobr></th>,
 <th class="text-center"><nobr>УИК №2086</nobr></th>,
 <th class="text-center"><nobr>УИК №2087</nobr></th>,
 <th class="text-center"><nobr>УИК №2088</nobr></th>,
 <th class="text-center"><nobr>УИК №2089</nobr></th>,
 <th class="text-center"><nobr>УИК №2090</nobr></th>,
 <th class="text-center"><nobr>УИК №2091</nobr></th>,
 <th class="text-center"><nobr>УИК №2092</nobr></th>,
 <th class="text-center"><nobr>УИК №2093</nobr></th>,
 <th class="text-center"><nobr>УИК №2094</nobr></th>,
 <t

In [19]:
print(re.findall(r'УИК', UIK[0].text))
print(re.findall(r'УИК', UIK[1].text))

[]
['УИК']


In [20]:
UIK_names = []
for i in UIK:
    if len(re.findall(r'УИК', i.text)) != 0:
        UIK_names.append(i.text)
print(UIK_names)

['УИК №2078', 'УИК №2079', 'УИК №2080', 'УИК №2081', 'УИК №2082', 'УИК №2083', 'УИК №2084', 'УИК №2085', 'УИК №2086', 'УИК №2087', 'УИК №2088', 'УИК №2089', 'УИК №2090', 'УИК №2091', 'УИК №2092', 'УИК №2093', 'УИК №2094', 'УИК №2095', 'УИК №2096', 'УИК №2097', 'УИК №2098', 'УИК №2099', 'УИК №2100', 'УИК №2101', 'УИК №2102', 'УИК №2103', 'УИК №2104', 'УИК №2105', 'УИК №2106', 'УИК №2107', 'УИК №2108', 'УИК №2109', 'УИК №2110', 'УИК №2111', 'УИК №2112', 'УИК №2113', 'УИК №2114', 'УИК №2115', 'УИК №2116', 'УИК №2117', 'УИК №3736']


In [22]:
# попробуем вытащить все цифры
UIK_data = soup0.find_all('td', {'class':'text-right'})
#UIK_data

In [23]:
UIK_dict = {} # ключ - название УИКа из UIK_names; значение - список с данными

for i in range(len(UIK_names)):  # для каждого элемента в UIK_names (для каждого УИКа)
    
    ind = i                      # задаем, что первый индекс (счетчик) = индексу названия УИКа в UIK_names
    x = 1                        # задаем счетчик, который скажет нам, когда нужно остановиться и перейти к 
                                 # следующему УИКу
        
    UIK_dict[UIK_names[i]] = []  # для каждого сразу создаем значение словаря - пустой список, 
                                 # куда потом добавляем данные
    
    while x <= len(rows):        # пока количество данных, не превышает допустимое 
                                 #(будет странно, если всего строк 32, а мы получим 50 данных) 
            
        #print(i, ind, x)        # если убрать эти комментарии, можно следить за тем, как работает программа
        #print(UIK_names[i], UIK_data[ind])
        
        UIK_dict[UIK_names[i]].append(UIK_data[ind]) # добавляем данные к соответствующему УИКу
        ind += len(UIK_names)    # говорим, что следующий индекс будет больше на число количества УИКов
        x += 1                   # считаем итерации цикла (которые сравниваем с количеством строк)
        
        
            

Но мы вытащили не совсем числа, а их запись в тегах. Посмотрим на то, как можно преобразовать ее с помощью регулярных выражений. Заметим, что там два возможных типа записи — просто число и число с указанием процента. Нам нужно вытащить и то, и то, желательно очень компактно.

In [24]:
UIK_dict['УИК №2078'][0].text

'2314\n\n'

In [25]:
# указываем, что может идти любой набор чисел, а также может быть точка и %
re.findall(r'[0-9\.%]+', UIK_dict['УИК №2078'][0].text)  

['2314']

In [26]:
UIK_dict['УИК №2078'][-1].text

'46\n5.66%\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t'

In [27]:
re.findall(r'[0-9\.%]+',  UIK_dict['УИК №2078'][-1].text)

['46', '5.66%']

Напишем функцию, которая будет обрабатывать данные: 

In [30]:
def UIK_data_func(i): # i типа <td class="text-right"><nobr><b>2314</b></nobr> <br/></td>
    i = re.findall(r'[0-9\.%]+',  i.text) 
    if len(i) > 1:
        return ' '.join([(i[0]), '('+i[1]+')']) 
    else:
        return int(i[0]) # добавим, что может быть int

In [31]:
UIK_data_func(UIK_dict['УИК №2078'][-1])

'46 (5.66%)'

Теперь перепишем код с нашим словарем с УИКами: 

In [32]:
UIK_dict = {} 

for i in range(len(UIK_names)):  
    
    ind = i                      
    x = 1                        
        
    UIK_dict[UIK_names[i]] = []   
                                 
    
    while x <= len(rows):
        UIK_dict[UIK_names[i]].append(UIK_data_func(UIK_data[ind])) # вот тут применяем функцию!
        ind += len(UIK_names)    
        x += 1        

In [33]:
print(UIK_dict['УИК №2078'])

[2314, 1600, 0, 791, 22, 787, 22, 791, 5, 808, 30, 16, 4, 14, 0, 0, 0, 0, '13 (1.60%)', '9 (1.11%)', '4 (0.49%)', '478 (58.79%)', '2 (0.25%)', '1 (0.12%)', '88 (10.82%)', '15 (1.85%)', '17 (2.09%)', '2 (0.25%)', '46 (5.66%)', '77 (9.47%)', '10 (1.23%)', '46 (5.66%)']


In [34]:
# словарь словарей должен выглядеть вот так (это для одного р-на, который мы сейчас посмотрели) 
districts = {'район Чертаново Южное': UIK_dict}
#districts

А теперь делаем наш словарь автоматичеки: 

In [35]:
districts = {}

for j in soup.find_all('a', {'href' : re.compile('region')}):
    if len(re.findall(r'район', j.text)) != 0:
        if j.text not in districts:
            
            #### Работа с сылкой
            url0 = j.get('href')
            page0 = requests.get(url0)
            soup0 = BeautifulSoup(page0.text)
            sleep(1) # не забываем спать!
            
            UIK = soup0.find_all('th', {'class':'text-center'})
            
            #### Работа с УИКами
            
            UIK_names = []
            for i in UIK:
                if len(re.findall(r'УИК', i.text)) != 0:
                    UIK_names.append(i.text)
                    
            UIK_data = soup0.find_all('td', {'class':'text-right'})
            
            UIK_dict = {} 

            for i in range(len(UIK_names)):  

                ind = i                      
                x = 1                        

                UIK_dict[UIK_names[i]] = []   


                while x <= len(rows):
                    UIK_dict[UIK_names[i]].append(UIK_data_func(UIK_data[ind])) # вот тут применяем функцию!
                    ind += len(UIK_names)    
                    x += 1      
            
            districts[j.text] = UIK_dict

In [ ]:
#districts # похоже на правду

Осталось оформить все в таблицу!

In [132]:
df = pd.DataFrame(districts['район Северное Бутово'], index = rows)
df

,УИК №2315,УИК №2316,УИК №2317,УИК №2318,УИК №2319,УИК №2320,УИК №2321,УИК №2322,УИК №2323,УИК №2324,...,УИК №2332,УИК №2333,УИК №2334,УИК №2335,УИК №2336,УИК №2337,УИК №2338,УИК №2339,УИК №2340,УИК №2341
"Число избирателей, внесенных в список избирателей на момент окончания голосования",2459,2550,2035,2061,2457,2097,2264,2253,2563,2118,...,2222,735,2318,2374,2141,2510,2463,2464,2262,2548
"Число избирательных бюллетеней, полученных участковой избирательной комиссией",2000,1900,1600,1600,2000,1700,1800,1800,2000,1700,...,1800,600,1900,1900,1700,2000,2000,2000,1800,2000
"Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Число избирательных бюллетеней, выданных в помещении для голосования в день голосования",754,783,707,669,815,641,740,794,844,688,...,727,216,671,761,687,758,774,809,720,765
"Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования",18,13,12,11,21,28,14,13,8,13,...,5,6,18,22,15,12,20,18,30,4
Число погашенных избирательных бюллетеней,1228,1104,881,920,1164,1031,1046,993,1148,999,...,1068,378,1211,1117,998,1230,1206,1173,1050,1231
"Число избирательных бюллетеней, содержащихся в переносных ящиках для голосования",18,13,12,11,21,28,14,13,8,13,...,5,6,18,22,15,12,20,18,30,4
"Число избирательных бюллетеней, содержащихся в стационарных ящиках для голосования",754,781,707,669,812,641,737,781,836,686,...,727,214,669,761,687,758,773,809,719,765
Число недействительных избирательных бюллетеней,18,11,16,10,14,20,17,15,7,14,...,10,3,19,14,23,15,20,15,16,10
Число действительных избирательных бюллетеней,754,783,703,670,819,649,734,779,837,685,...,722,217,668,769,679,755,773,812,733,759


Сделаем двухуровневый заголовок:

In [140]:
df.columns = [['район Северное Бутово']*len(districts['район Северное Бутово'].keys()),
                            list(districts['район Северное Бутово'].keys())]

In [141]:
df

район Северное Бутово  \
                                                               УИК №2315   
Число избирателей, внесенных в список избирател...                  2459   
Число избирательных бюллетеней, полученных учас...                  2000   
Число избирательных бюллетеней, выданных избира...                     0   
Число избирательных бюллетеней, выданных в поме...                   754   
Число избирательных бюллетеней, выданных вне по...                    18   
Число погашенных избирательных бюллетеней                           1228   
Число избирательных бюллетеней, содержащихся в ...                    18   
Число избирательных бюллетеней, содержащихся в ...                   754   
Число недействительных избирательных бюллетеней                       18   
Число действительных избирательных бюллетеней                        754   
Число открепительных удостоверений, полученных ...                    20   
Число открепительных удостоверений, выданных на...                    12   
Число избирателей, проголосовавших по открепите...                    11   
Число погашенных неиспользованных открепительны...                     8   
Число открепительных удостоверений, выданных из...                     0   
Число утраченных открепительных удостоверений                          0   
Число утраченных избирательных бюллетеней                              0   
Число избирательных бюллетеней, не учтенных при...                     0   
ВСЕРОССИЙСКАЯ ПОЛИТИЧЕСКАЯ ПАРТИЯ "РОДИНА"                    36 (4.66%)   
Политическая партия КОММУНИСТИЧЕСКАЯ ПАРТИЯ КОМ...            14 (1.81%)   
Политическая партия "Российская партия пенсионе...            22 (2.85%)   
Всероссийская политическая партия "ЕДИНАЯ РОССИЯ"           304 (39.38%)   
Политическая партия "Российская экологическая п...            19 (2.46%)   
Политическая партия "Гражданская Платформа"                    1 (0.13%)   
Политическая партия ЛДПР - Либерально-демократи...          106 (13.73%)   
Политическая партия "Партия народной свободы" (...            14 (1.81%)   
Всероссийская политическая партия "ПАРТИЯ РОСТА"              30 (3.89%)   
Общественная организация Всероссийская политиче...             1 (0.13%)   
Политическая партия "Российская объединенная де...            46 (5.96%)   
Политическая партия "КОММУНИСТИЧЕСКАЯ ПАРТИЯ РО...           97 (12.56%)   
Политическая партия "ПАТРИОТЫ РОССИИ"                          6 (0.78%)   
Политическая партия СПРАВЕДЛИВАЯ РОССИЯ                       58 (7.51%)   

                                                                  \
                                                       УИК №2316   
Число избирателей, внесенных в список избирател...          2550   
Число избирательных бюллетеней, полученных учас...          1900   
Число избирательных бюллетеней, выданных избира...             0   
Число избирательных бюллетеней, выданных в поме...           783   
Число избирательных бюллетеней, выданных вне по...            13   
Число погашенных избирательных бюллетеней                   1104   
Число избирательных бюллетеней, содержащихся в ...            13   
Число избирательных бюллетеней, содержащихся в ...           781   
Число недействительных избирательных бюллетеней               11   
Число действительных избирательных бюллетеней                783   
Число открепительных удостоверений, полученных ...            20   
Число открепительных удостоверений, выданных на...            16   
Число избирателей, проголосовавших по открепите...             4   
Число погашенных неиспользованных открепительны...             4   
Число открепительных удостоверений, выданных из...             4   
Число утраченных открепительных удостоверений                  0   
Число утраченных избирательных бюллетеней                      0   
Число избирательных бюллетеней, не учтенных при...             0   
ВСЕРОССИЙСКАЯ ПОЛИТИЧЕСКАЯ ПАРТИЯ "РОДИНА"            28 (3.53%)   
Политическая партия КОММУНИСТИЧЕСКАЯ П

Попытка ответить на вопрос, можно ли подписать "район Северное Бутово" над каждым УИКом. К сожалению, нет, но можер попробовать сделат так: 

In [144]:
#df.columns = df.columns.swaplevel(0, 1) # меняет уровни местами
#df

район Северное Бутово  \
                                                               УИК №2315   
Число избирателей, внесенных в список избирател...                  2459   
Число избирательных бюллетеней, полученных учас...                  2000   
Число избирательных бюллетеней, выданных избира...                     0   
Число избирательных бюллетеней, выданных в поме...                   754   
Число избирательных бюллетеней, выданных вне по...                    18   
Число погашенных избирательных бюллетеней                           1228   
Число избирательных бюллетеней, содержащихся в ...                    18   
Число избирательных бюллетеней, содержащихся в ...                   754   
Число недействительных избирательных бюллетеней                       18   
Число действительных избирательных бюллетеней                        754   
Число открепительных удостоверений, полученных ...                    20   
Число открепительных удостоверений, выданных на...                    12   
Число избирателей, проголосовавших по открепите...                    11   
Число погашенных неиспользованных открепительны...                     8   
Число открепительных удостоверений, выданных из...                     0   
Число утраченных открепительных удостоверений                          0   
Число утраченных избирательных бюллетеней                              0   
Число избирательных бюллетеней, не учтенных при...                     0   
ВСЕРОССИЙСКАЯ ПОЛИТИЧЕСКАЯ ПАРТИЯ "РОДИНА"                    36 (4.66%)   
Политическая партия КОММУНИСТИЧЕСКАЯ ПАРТИЯ КОМ...            14 (1.81%)   
Политическая партия "Российская партия пенсионе...            22 (2.85%)   
Всероссийская политическая партия "ЕДИНАЯ РОССИЯ"           304 (39.38%)   
Политическая партия "Российская экологическая п...            19 (2.46%)   
Политическая партия "Гражданская Платформа"                    1 (0.13%)   
Политическая партия ЛДПР - Либерально-демократи...          106 (13.73%)   
Политическая партия "Партия народной свободы" (...            14 (1.81%)   
Всероссийская политическая партия "ПАРТИЯ РОСТА"              30 (3.89%)   
Общественная организация Всероссийская политиче...             1 (0.13%)   
Политическая партия "Российская объединенная де...            46 (5.96%)   
Политическая партия "КОММУНИСТИЧЕСКАЯ ПАРТИЯ РО...           97 (12.56%)   
Политическая партия "ПАТРИОТЫ РОССИИ"                          6 (0.78%)   
Политическая партия СПРАВЕДЛИВАЯ РОССИЯ                       58 (7.51%)   

                                                                  \
                                                       УИК №2316   
Число избирателей, внесенных в список избирател...          2550   
Число избирательных бюллетеней, полученных учас...          1900   
Число избирательных бюллетеней, выданных избира...             0   
Число избирательных бюллетеней, выданных в поме...           783   
Число избирательных бюллетеней, выданных вне по...            13   
Число погашенных избирательных бюллетеней                   1104   
Число избирательных бюллетеней, содержащихся в ...            13   
Число избирательных бюллетеней, содержащихся в ...           781   
Число недействительных избирательных бюллетеней               11   
Число действительных избирательных бюллетеней                783   
Число открепительных удостоверений, полученных ...            20   
Число открепительных удостоверений, выданных на...            16   
Число избирателей, проголосовавших по открепите...             4   
Число погашенных неиспользованных открепительны...             4   
Число открепительных удостоверений, выданных из...             4   
Число утраченных открепительных удостоверений                  0   
Число утраченных избирательных бюллетеней                      0   
Число избирательных бюллетеней, не учтенных при...             0   
ВСЕРОССИЙСКАЯ ПОЛИТИЧЕСКАЯ ПАРТИЯ "РОДИНА"            28 (3.53%)   
Политическая партия КОММУНИСТИЧЕСКАЯ П

Добавим остальные районы: 

In [145]:
for i in list(districts.keys())[1:]:
    df_to_join = pd.DataFrame(districts[i], index = rows)
    df_to_join.columns = [[i]*len(districts[i].keys()),
                            list(districts[i].keys())]
    
    df = df.join(df_to_join)

In [146]:
df#.transpose()
#df.to_excel('data.xlsx')

район Северное Бутово  \
                                                               УИК №2315   
Число избирателей, внесенных в список избирател...                  2459   
Число избирательных бюллетеней, полученных учас...                  2000   
Число избирательных бюллетеней, выданных избира...                     0   
Число избирательных бюллетеней, выданных в поме...                   754   
Число избирательных бюллетеней, выданных вне по...                    18   
Число погашенных избирательных бюллетеней                           1228   
Число избирательных бюллетеней, содержащихся в ...                    18   
Число избирательных бюллетеней, содержащихся в ...                   754   
Число недействительных избирательных бюллетеней                       18   
Число действительных избирательных бюллетеней                        754   
Число открепительных удостоверений, полученных ...                    20   
Число открепительных удостоверений, выданных на...                    12   
Число избирателей, проголосовавших по открепите...                    11   
Число погашенных неиспользованных открепительны...                     8   
Число открепительных удостоверений, выданных из...                     0   
Число утраченных открепительных удостоверений                          0   
Число утраченных избирательных бюллетеней                              0   
Число избирательных бюллетеней, не учтенных при...                     0   
ВСЕРОССИЙСКАЯ ПОЛИТИЧЕСКАЯ ПАРТИЯ "РОДИНА"                    36 (4.66%)   
Политическая партия КОММУНИСТИЧЕСКАЯ ПАРТИЯ КОМ...            14 (1.81%)   
Политическая партия "Российская партия пенсионе...            22 (2.85%)   
Всероссийская политическая партия "ЕДИНАЯ РОССИЯ"           304 (39.38%)   
Политическая партия "Российская экологическая п...            19 (2.46%)   
Политическая партия "Гражданская Платформа"                    1 (0.13%)   
Политическая партия ЛДПР - Либерально-демократи...          106 (13.73%)   
Политическая партия "Партия народной свободы" (...            14 (1.81%)   
Всероссийская политическая партия "ПАРТИЯ РОСТА"              30 (3.89%)   
Общественная организация Всероссийская политиче...             1 (0.13%)   
Политическая партия "Российская объединенная де...            46 (5.96%)   
Политическая партия "КОММУНИСТИЧЕСКАЯ ПАРТИЯ РО...           97 (12.56%)   
Политическая партия "ПАТРИОТЫ РОССИИ"                          6 (0.78%)   
Политическая партия СПРАВЕДЛИВАЯ РОССИЯ                       58 (7.51%)   

                                                                  \
                                                       УИК №2316   
Число избирателей, внесенных в список избирател...          2550   
Число избирательных бюллетеней, полученных учас...          1900   
Число избирательных бюллетеней, выданных избира...             0   
Число избирательных бюллетеней, выданных в поме...           783   
Число избирательных бюллетеней, выданных вне по...            13   
Число погашенных избирательных бюллетеней                   1104   
Число избирательных бюллетеней, содержащихся в ...            13   
Число избирательных бюллетеней, содержащихся в ...           781   
Число недействительных избирательных бюллетеней               11   
Число действительных избирательных бюллетеней                783   
Число открепительных удостоверений, полученных ...            20   
Число открепительных удостоверений, выданных на...            16   
Число избирателей, проголосовавших по открепите...             4   
Число погашенных неиспользованных открепительны...             4   
Число открепительных удостоверений, выданных из...             4   
Число утраченных открепительных удостоверений                  0   
Число утраченных избирательных бюллетеней                      0   
Число избирательных бюллетеней, не учтенных при...             0   
ВСЕРОССИЙСКАЯ ПОЛИТИЧЕСКАЯ ПАРТИЯ "РОДИНА"            28 (3.53%)   
Политическая партия КОММУНИСТИЧЕСКАЯ П

In [147]:
df['район Северное Бутово']['УИК №2315']

Число избирателей, внесенных в список избирателей на момент окончания голосования                           2459
Число избирательных бюллетеней, полученных участковой избирательной комиссией                               2000
Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно                                 0
Число избирательных бюллетеней, выданных в помещении для голосования в день голосования                      754
Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования                     18
Число погашенных избирательных бюллетеней                                                                   1228
Число избирательных бюллетеней, содержащихся в переносных ящиках для голосования                              18
Число избирательных бюллетеней, содержащихся в стационарных ящиках для голосования                           754
Число недействительных избирательных бюллетеней                                                 

### Задание 1

Дана ссылка на сайт [IMDb](https://www.imdb.com/calendar/?ref_=nv_mv_cal). Сохраните название и ссылки всех фильмов на странице. 

In [148]:
url = 'https://www.imdb.com/calendar/?ref_=nv_mv_cal'
print(url)

https://www.imdb.com/calendar/?ref_=nv_mv_cal


In [149]:
page = requests.get(url)
page.status_code # 200 - все хорошо

200

In [150]:
soup = BeautifulSoup(page.text)
#soup

In [151]:
dates = soup.find_all('h4') # все наши даты
dates

[<h4>28 May 2021</h4>,
 <h4>01 June 2021</h4>,
 <h4>04 June 2021</h4>,
 <h4>08 June 2021</h4>,
 <h4>11 June 2021</h4>,
 <h4>16 June 2021</h4>,
 <h4>18 June 2021</h4>,
 <h4>25 June 2021</h4>,
 <h4>30 June 2021</h4>,
 <h4>02 July 2021</h4>,
 <h4>09 July 2021</h4>,
 <h4>16 July 2021</h4>,
 <h4>20 July 2021</h4>,
 <h4>23 July 2021</h4>,
 <h4>27 July 2021</h4>,
 <h4>30 July 2021</h4>,
 <h4>06 August 2021</h4>,
 <h4>13 August 2021</h4>,
 <h4>20 August 2021</h4>,
 <h4>27 August 2021</h4>,
 <h4>03 September 2021</h4>,
 <h4>10 September 2021</h4>,
 <h4>24 September 2021</h4>,
 <h4>01 October 2021</h4>,
 <h4>08 October 2021</h4>,
 <h4>15 October 2021</h4>,
 <h4>22 October 2021</h4>,
 <h4>29 October 2021</h4>,
 <h4>05 November 2021</h4>,
 <h4>11 November 2021</h4>,
 <h4>19 November 2021</h4>,
 <h4>24 November 2021</h4>,
 <h4>03 December 2021</h4>,
 <h4>10 December 2021</h4>,
 <h4>17 December 2021</h4>,
 <h4>22 December 2021</h4>,
 <h4>07 January 2022</h4>,
 <h4>14 January 2022</h4>,
 <h4>28 Janua

In [152]:
soup.find_all('h4')[0].text

'28 May 2021'

In [153]:
# достать ссылки можно так, но как понять, к какой дате они относятся??
soup.find_all('a', attrs = {'href' : re.compile('title/tt')}) 

[<a href="/title/tt3228774/?ref_=rlm">Cruella</a>,
 <a href="/title/tt8332922/?ref_=rlm">A Quiet Place Part II</a>,
 <a href="/title/tt7050946/?ref_=rlm">American Traitor: The Trial of Axis Sally</a>,
 <a href="/title/tt13863334/?ref_=rlm">Endangered Species</a>,
 <a href="/title/tt6852534/?ref_=rlm">Funhouse</a>,
 <a href="/title/tt11666128/?ref_=rlm">Potter's Ground</a>,
 <a href="/title/tt7069210/?ref_=rlm">The Conjuring: The Devil Made Me Do It</a>,
 <a href="/title/tt11084896/?ref_=rlm">Spirit Untamed</a>,
 <a href="/title/tt8372094/?ref_=rlm">Flashback</a>,
 <a href="/title/tt6384920/?ref_=rlm">Under the Stadium Lights</a>,
 <a href="/title/tt5013984/?ref_=rlm">Gully</a>,
 <a href="/title/tt1318523/?ref_=rlm">The Unhealer</a>,
 <a href="/title/tt1321510/?ref_=rlm">In the Heights</a>,
 <a href="/title/tt9733388/?ref_=rlm">Sublet</a>,
 <a href="/title/tt10329614/?ref_=rlm">Censor</a>,
 <a href="/title/tt4876134/?ref_=rlm">The Misfits</a>,
 <a href="/title/tt8338076/?ref_=rlm">Queen

In [155]:
# пробуем по-другому
info = soup.find_all('div', {'id': 'main'})[0].find_all('ul')
#info

Теперь количество дат соответствует количеству элементов полученного выше списка.

In [156]:
info[0]

<ul>
<li>
<a href="/title/tt3228774/?ref_=rlm">Cruella</a> (2021)
                        </li>
<li>
<a href="/title/tt8332922/?ref_=rlm">A Quiet Place Part II</a> (2020)
                        </li>
<li>
<a href="/title/tt7050946/?ref_=rlm">American Traitor: The Trial of Axis Sally</a> (2021)
                        </li>
<li>
<a href="/title/tt13863334/?ref_=rlm">Endangered Species</a> (2021)
                        </li>
<li>
<a href="/title/tt6852534/?ref_=rlm">Funhouse</a> (2019)
                        </li>
</ul>

In [157]:
info[0].find_all('a')[0]

<a href="/title/tt3228774/?ref_=rlm">Cruella</a>

In [158]:
'https://www.imdb.com' + info[0].find_all('a')[0].get('href')

'https://www.imdb.com/title/tt3228774/?ref_=rlm'

In [159]:
urls = ['https://www.imdb.com' + i.get('href') for i in info[0].find_all('a')]
urls # это ссылки на все фильмы соответствующей даты

['https://www.imdb.com/title/tt3228774/?ref_=rlm',
 'https://www.imdb.com/title/tt8332922/?ref_=rlm',
 'https://www.imdb.com/title/tt7050946/?ref_=rlm',
 'https://www.imdb.com/title/tt13863334/?ref_=rlm',
 'https://www.imdb.com/title/tt6852534/?ref_=rlm']

In [160]:
info[0].find_all('a')[0].text

'Cruella'

In [161]:
names = [i.text for i in info[0].find_all('a')]
names

['Cruella',
 'A Quiet Place Part II',
 'American Traitor: The Trial of Axis Sally',
 'Endangered Species',
 'Funhouse']

In [164]:
movies = {}
for d in range(len(dates)):
    
    names = [i.text for i in info[d].find_all('a')]
    urls = ['https://www.imdb.com' + i.get('href') for i in info[d].find_all('a')]
    
    urls_names = {}
    for x in range(len(names)):
        urls_names[names[x]] = [urls[x]] # ключом словаря будет дата, а значением — ссылка
    
    movies[dates[d].text] = urls_names
#movies

In [165]:
rows = ['Ссылка']
df = pd.DataFrame(movies['28 May 2021'], index = rows)
df

,Cruella,A Quiet Place Part II,American Traitor: The Trial of Axis Sally,Endangered Species,Funhouse
Ссылка,https://www.imdb.com/title/tt3228774/?ref_=rlm,https://www.imdb.com/title/tt8332922/?ref_=rlm,https://www.imdb.com/title/tt7050946/?ref_=rlm,https://www.imdb.com/title/tt13863334/?ref_=rlm,https://www.imdb.com/title/tt6852534/?ref_=rlm


In [166]:
df.columns = [['28 May 2021']*len(movies['28 May 2021'].keys()),
                            list(movies['28 May 2021'].keys())]

In [168]:
for i in list(movies.keys())[1:]:
    df_to_join = pd.DataFrame(movies[i], index = rows)
    df_to_join.columns = [[i]*len(movies[i].keys()),
                            list(movies[i].keys())]
    
    df = df.join(df_to_join)

In [169]:
df.transpose()

Ссылка
28 May 2021   Cruella                                     https://www.imdb.com/title/tt3228774/?ref_=rlm
              A Quiet Place Part II                       https://www.imdb.com/title/tt8332922/?ref_=rlm
              American Traitor: The Trial of Axis Sally   https://www.imdb.com/title/tt7050946/?ref_=rlm
              Endangered Species                         https://www.imdb.com/title/tt13863334/?ref_=rlm
              Funhouse                                    https://www.imdb.com/title/tt6852534/?ref_=rlm
...                                                                                                  ...
08 April 2022 Sonic the Hedgehog 2                       https://www.imdb.com/title/tt12412888/?ref_=rlm
15 April 2022 Lost City of D                             https://www.imdb.com/title/tt13320622/?ref_=rlm
06 May 2022   Thor: Love and Thunder                     https://www.imdb.com/title/tt10648342/?ref_=rlm
27 May 2022   Mission: Impossible 7                       https://www.imdb.com/title/tt9603212/?ref_=rlm
              John Wick: Chapter 4                       https://www.imdb.com/title/tt10366206/?ref_=rlm

[100 rows x 1 columns]

### Задание 2

Соберите информацию о сотрудниках [кафедры высшей математики](https://hmat.hse.ru/persons). Если сотрудник ведет какие-то курсы, то также название курса и ссылка на его страницу.

In [172]:
url = 'https://hmat.hse.ru/persons'
page = requests.get(url)
page.status_code

200

In [173]:
soup = BeautifulSoup(page.text)
#soup

In [174]:
info = soup.find_all('div', {'class': 'fa-person__box'})
#info

In [175]:
info[14]

<div class="fa-person__box">
<a class="fa-person__name" href="//www.hse.ru/org/persons/207912918">
							Латышев Александр Михайлович
						</a>
<p class="fa-person__info">Старший преподаватель</p>
</div>

In [176]:
name = info[14].find('a').text.strip()
name

'Латышев Александр Михайлович'

In [177]:
degree = info[14].find('p').text.strip()
degree

'Старший преподаватель'

In [178]:
url_person = 'https:' + info[14].find('a').get('href')
url_person

'https://www.hse.ru/org/persons/207912918'

In [179]:
url0 = 'https://www.hse.ru/org/persons/207912918'
url0

'https://www.hse.ru/org/persons/207912918'

In [180]:
page0 = requests.get(url0)
page0.status_code

200

In [181]:
soup0 = BeautifulSoup(page0.text)
#soup0

In [182]:
courses = soup0.find_all('div', {'tab-node':'edu-courses'})
courses

[<div class="b-person-data" tab-index="teaching" tab-node="edu-courses"><a id="m-teaching"></a><h3>Учебные курсы (2020/2021 уч. год)</h3><div class="b-person-data__inner"><a class="link" href="https://www.hse.ru/edu/courses/304869606">Математический анализ</a> (Бакалавриат; где читается: <a class="link" href="https://chemistry.hse.ru/">Факультет химии</a>; 2-й курс, 1-4 модуль)<span class="language-label">Рус</span></div></div>]

In [183]:
courses_info = courses[0].find_all('a', {'class':'link'})
courses_info

[<a class="link" href="https://www.hse.ru/edu/courses/304869606">Математический анализ</a>,
 <a class="link" href="https://chemistry.hse.ru/">Факультет химии</a>]

In [184]:
staff = {}

for i in info:
    name = i.find('a').text.strip()
    degree = i.find('p').text.strip()
    url_person = 'https:' + i.find('a').get('href')
    
    page0 = requests.get(url_person)
    soup0 = BeautifulSoup(page0.text)
    
    sleep(1)
    
    courses = soup0.find_all('div', {'tab-node':'edu-courses'})
    if len(courses) != 0:
        courses_info = courses[0].find_all('a', {'class':'link'})

        empty_dict = {}

        for i in range(0, len(courses_info), 2):

            key = courses_info[i].get('href')
            value_name = courses_info[i].text
            value_fac = courses_info[i+1].text

            empty_dict[key] = [value_name, value_fac]

        staff[f'{name}({degree})'] = empty_dict
    

In [185]:
rows = ['Название', 'Факультет']
df = pd.DataFrame(staff[list(staff.keys())[0]], index = rows)

df.columns = [[list(staff.keys())[0]]*len(staff[list(staff.keys())[0]].keys()),
                            list(staff[list(staff.keys())[0]].keys())]

df

Макаров Алексей Алексеевич(Профессор)  \
                    https://www.hse.ru/edu/courses/384753543   
Название   Введение в теорию вероятностей и математическу...   
Факультет                          Факультет социальных наук   

                                                    \
          https://www.hse.ru/edu/courses/389913932   
Название                                Математика   
Факультет                Факультет социальных наук   

                                                              \
                    https://www.hse.ru/edu/courses/386267292   
Название   Математические и статистические методы в психо...   
Факультет                          Факультет социальных наук   

                                                            \
                  https://www.hse.ru/edu/courses/375287963   
Название   Теория вероятностей и математическая статистика   
Факультет                        Факультет социальных наук   

                                                            
                  https://www.hse.ru/edu/courses/389892476  
Название   Теория вероятностей и математическая статистика  
Факультет                        Факультет социальных наук

In [186]:
for i in list(staff.keys())[1:]:
    df_to_join = pd.DataFrame(staff[i], index = rows)
    df_to_join.columns = [[i]*len(staff[i].keys()),
                            list(staff[i].keys())]
    
    df = df.join(df_to_join)

In [188]:
df.transpose()

Название  \
Макаров Алексей Алексеевич(Профессор) https://www.hse.ru/edu/courses/384753543  Введение в теорию вероятностей и математическу...   
                                      https://www.hse.ru/edu/courses/389913932                                         Математика   
                                      https://www.hse.ru/edu/courses/386267292  Математические и статистические методы в психо...   
                                      https://www.hse.ru/edu/courses/375287963    Теория вероятностей и математическая статистика   
                                      https://www.hse.ru/edu/courses/389892476    Теория вероятностей и математическая статистика   
...                                                                                                                           ...   
Шеремет Елизавета Павловна(Ассистент) https://www.hse.ru/edu/courses/339569771  Научно-исследовательский семинар "Основы научн...   
                                      https://www.hse.ru/edu/courses/375276621         Основы прикладной математики и информатики   
                                      https://www.hse.ru/edu/courses/375302503  Современная социологическая теория: модели объ...   
                                      https://www.hse.ru/edu/courses/339565162                             Социологическая теория   
                                      https://www.hse.ru/edu/courses/383210134    Теория вероятностей и математическая статистика   

                                                                                                              Факультет  
Макаров Алексей Алексеевич(Профессор) https://www.hse.ru/edu/courses/384753543                Факультет социальных наук  
                                      https://www.hse.ru/edu/courses/389913932                Факультет социальных наук  
                                      https://www.hse.ru/edu/courses/386267292                Факультет социальных наук  
                                      https://www.hse.ru/edu/courses/375287963                Факультет социальных наук  
                                      https://www.hse.ru/edu/courses/389892476                Факультет социальных наук  
...                                                                                                                 ...  
Шеремет Елизавета Павловна(Ассистент) https://www.hse.ru/edu/courses/339569771  Факультет коммуникаций, медиа и дизайна  
                                      https://www.hse.ru/edu/courses/375276621  Факультет коммуникаций, медиа и дизайна  
                                      https://www.hse.ru/edu/courses/375302503                Факультет социальных наук  
                                      https://www.hse.ru/edu/courses/339565162                Факультет социальных наук  
                                      https://www.hse.ru/edu/courses/383210134                     Высшая школа бизнеса  

[91 rows x 2 columns]

### Задание 3

Дана ссылка на первую страницу [каталога книжных новинок](https://www.bgshop.ru/catalog/group?id=444&page=1&sort=1&instock=). Соберите информацию о книгах с первых трех страниц. 

In [189]:
urls = []

for i in range(1,4):
    urls.append(f'https://www.bgshop.ru/catalog/group?id=444&page={i}&sort=1&instock=')
urls

['https://www.bgshop.ru/catalog/group?id=444&page=1&sort=1&instock=',
 'https://www.bgshop.ru/catalog/group?id=444&page=2&sort=1&instock=',
 'https://www.bgshop.ru/catalog/group?id=444&page=3&sort=1&instock=']

In [190]:
page = requests.get(urls[0])
page.status_code

200

In [191]:
soup = BeautifulSoup(page.text)
#soup

In [192]:
product = soup.find_all('div', {'class':'product'})[0]
product

<div class="product">
<a class="img_link" href="/Catalog/GetFullDescription?id=10767952&amp;type=1"><img alt="" class="img-fluid" src="https://static1.bgshop.ru/imagehandler.ashx?fileName=10767952.jpg&amp;width=200"/></a>
<div class="text">
<div class="author">Дмитрий Быков.</div>
<h3><a href="/Catalog/GetFullDescription?id=10767952&amp;type=1" id="p_title_10767952">Истребитель : роман</a></h3>
<p class="price">
<span class="im_price_title" style="font-size:0.85em;">Цена в интернет-магазине: </span><span id="p_price_10767952" style="display:block; font-weight:bold; font-size:1.4em; margin:6px 0 5px; color:#111;">
                                792,00 <span style="font-size:.8em; font-weight:normal;">руб.</span>
</span>
<span style="display:block;"> </span>
</p>
<p class="status im_status_title">
в наличии                        </p>
<p class="buttons">
<span class="btn btn-primary" id="add_10767952" onclick="AddToBasket(10767952)"><i class="fa fa-shopping-cart"></i> В корзину</span>
<

In [193]:
# ссылка
url0 = 'https://www.bgshop.ru/' + product.find_all('a')[0].get('href')
url0

'https://www.bgshop.ru//Catalog/GetFullDescription?id=10767952&type=1'

In [194]:
page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text)
#soup0

In [195]:
main_info = soup0.find_all('div', {'id':'productMain'})[0]
main_info

<div class="row box" id="productMain">
<div class="col-md-4">
<a data-fancybox="gallery" href="/imagehandler.ashx?filename=10767952.jpg&amp;width=400"><img alt="Истребитель : роман" class="product_photo zoomIn" src="/imagehandler.ashx?filename=10767952.jpg&amp;width=200" title="Истребитель : роман" width="200"/></a>
</div>
<div class="col-md-7">
<div class="item-desc">
<p class="goToDescription"><a class="scroll-to" href="/Catalog/Search?query=Быков Д.Л.&amp;page=1">Быков Д.Л.</a></p>
<h1 id="p_title_10767952">Истребитель : роман</h1>
<p class="item-status">в наличии</p>
<div id="price"><b>Специальная цена</b> в интернет-магазине: <span id="p_price_10767952" style="color: #a42921; font-size:1.2rem;">792,00 руб.</span></div>
<p class="buttons">
<span class="btn btn-primary" id="add_10767952" onclick="AddToBasket(10767952)"><i class="fa fa-shopping-cart"></i> В корзину</span>
<a class="btn btn-primary add_but" href="/Basket/Detail" id="added_10767952">Перейти в корзину <i class="fa fa-ch

In [196]:
author = main_info.find_all('p')[0].text
author

'Быков Д.Л.'

In [197]:
title = main_info.find_all('h1')[0].text
title

'Истребитель : роман'

In [198]:
status = main_info.find_all('p', {'class':'item-status'})[0].text
status

'в наличии'

In [199]:
price = main_info.find_all('div', {'id':'price'})[0].text
price

'Специальная цена в интернет-магазине: 792,00\xa0руб.'

In [201]:
price_clean = float(re.findall(r'\d+,\d+', price)[0].replace(',', '.'))
price_clean

792.0

In [203]:
books = {}

for i in soup.find_all('div', {'class':'product'}):
    url0 = 'https://www.bgshop.ru/' + i.find_all('a')[0].get('href')
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text)
    sleep(1)
    
    main_info = soup0.find_all('div', {'id':'productMain'})[0]
    author = main_info.find_all('p')[0].text
    title = main_info.find_all('h1')[0].text
    status = main_info.find_all('p', {'class':'item-status'})[0].text
    
    price = main_info.find_all('div', {'id':'price'})[0].text
    price_clean = float(re.findall(r'\d+,\d+', price)[0].replace(',', '.'))
    
    books[title] = [author, status, price_clean, url0]

А теперь для всех (трех) страниц с книгами:

In [204]:
books = {}
for u in urls:
    page = requests.get(u)
    soup = BeautifulSoup(page.text)
    sleep(1)
    info = soup.find_all('div', {'class':'product'})
    for i in info:
        url0 = 'https://www.bgshop.ru/' + i.find_all('a')[0].get('href')
        page0 = requests.get(url0)
        soup0 = BeautifulSoup(page0.text)
        sleep(1)

        main_info = soup0.find_all('div', {'id':'productMain'})[0]
        author = main_info.find_all('p')[0].text
        title = main_info.find_all('h1')[0].text
        status = main_info.find_all('p', {'class':'item-status'})[0].text

        price = main_info.find_all('div', {'id':'price'})[0].text
        price_change = float(re.findall(r'\d+,\d+', price)[0].replace(',', '.'))

        books[title] = [author, status, price_change, url0]

In [205]:
df = pd.DataFrame(books, index = ['Автор', 'Статус', 'Цена', 'Ссылка']).transpose()
df

,Автор,Статус,Цена,Ссылка
Истребитель : роман,Быков Д.Л.,в наличии,792,https://www.bgshop.ru//Catalog/GetFullDescript...
Пешком по Москве 2,Жебрак М.,в наличии,449,https://www.bgshop.ru//Catalog/GetFullDescript...
"КРЫМ: Симферополь, Севастополь, Ялта, Феодосия, Керчь, Евпатория, Судак, Алушта, Бахчисарай, Тарханкут: путеводитель + карта",Лялюшина Ю.П.,в наличии,569,https://www.bgshop.ru//Catalog/GetFullDescript...
"Казахстан: Нур-Султан, Алматы и другие города республики",Якубова Н.И.,в наличии,306,https://www.bgshop.ru//Catalog/GetFullDescript...
Камчатка: путеводитель,Якубова Н.И.,в наличии,569,https://www.bgshop.ru//Catalog/GetFullDescript...
Москва глазами инженера,Багаутдинов А.,,609,https://www.bgshop.ru//Catalog/GetFullDescript...
Развиваем мозг. Японские секреты тренировки,Кавашима Р.,в наличии,381,https://www.bgshop.ru//Catalog/GetFullDescript...
"Ань, чего молчишь? Неосторожные шаги юности",Махлина А.Н.,в наличии,527,https://www.bgshop.ru//Catalog/GetFullDescript...
"Действуй как лидер, думай как лидер. Стратегический подход, который сделает вас сильным руководителем",Ибарра Э.,в наличии,569,https://www.bgshop.ru//Catalog/GetFullDescript...
Просто космос! Экспедиция на МКС с настоящим космонавтом,Усачёв Ю. В.,в наличии,449,https://www.bgshop.ru//Catalog/GetFullDescript...


### Задание 4

Дана ссылка на [сайт газеты "Московский комсомолец"](https://www.mk.ru/news/). Соберите ссылки на новости первой страницы, время публикации, заголовок. Пройдите по ссылкам и соберите количество просмотров и хэштеги статьи. 

In [206]:
url = 'https://www.mk.ru/news/'
page = requests.get(url)
soup = BeautifulSoup(page.text)

In [207]:
soup.find_all('li', {'class':'news-listing__item'})[0]

<li class="news-listing__item">
<a class="news-listing__item-link" href="https://www.mk.ru/politics/2021/05/28/putin-predlozhil-lukashenko-iskupatsya-v-sochinskom-more.html">
<span class="news-listing__item-time">18:20</span>
<h3 class="news-listing__item-title">Путин предложил Лукашенко искупаться в сочинском море</h3>
</a>
</li>

In [208]:
url0 = soup.find_all('li', {'class':'news-listing__item'})[0].find('a').get('href')
url0

'https://www.mk.ru/politics/2021/05/28/putin-predlozhil-lukashenko-iskupatsya-v-sochinskom-more.html'

In [209]:
time = soup.find_all('li', {'class':'news-listing__item'})[0].find('span').text
time

'18:20'

In [210]:
title = soup.find_all('li', {'class':'news-listing__item'})[0].find('h3').text
title

'Путин предложил Лукашенко искупаться в сочинском море'

In [211]:
page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text)

In [212]:
soup0.find_all('span', {'class':'meta__item_views'})[0].text.strip()

'363'

In [213]:
soup0.find('div', {'class':'article__tag'}).text

'\nВладимир Путин\nАлександр Лукашенко\nДмитрий Песков\nРоссия\nСочи\n'

In [214]:
re.findall(r'[^\n]+', soup0.find('div', {'class':'article__tag'}).text)

['Владимир Путин', 'Александр Лукашенко', 'Дмитрий Песков', 'Россия', 'Сочи']

In [215]:
news = {}
for i in soup.find_all('li', {'class':'news-listing__item'}):
    url0 = i.find('a').get('href')
    time = i.find('span').text
    title = i.find('h3').text
    
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text)
    sleep(1)
    
    view = int(soup0.find_all('span', {'class':'meta__item_views'})[0].text.strip())
    
    if soup0.find('div', {'class':'article__tag'}) is None:
        tags = ''
    else:
        tags = ', '.join(re.findall(r'[^\n]+', soup0.find('div', {'class':'article__tag'}).text))
        
    news[title] = [url0, time, view, tags]

In [223]:
# можно также сразу указывать, что у нас данные из словаря, orient='index' вместо transpose()
df = pd.DataFrame.from_dict(news, orient='index', columns = ['Ссылка','Время','Просмотры','Теги'])
df

,Ссылка,Время,Просмотры,Теги
Путин предложил Лукашенко искупаться в сочинском море,https://www.mk.ru/politics/2021/05/28/putin-pr...,18:20,363,"Владимир Путин, Александр Лукашенко, Дмитрий П..."
"Мясников рассказал об ""эпидемии"" на порядок смертоноснее коронавируса",https://www.mk.ru/social/2021/05/28/myasnikov-...,18:19,351,
Михаил Гуцериев показал Сергею Собянину новую школу в Новомосковском округе,https://www.mk.ru/social/2021/05/28/mikhail-gu...,18:18,200,"Сергей Собянин, Москва"
Мэр Риги прокомментировал свое уголовное дело в Белоруссии,https://www.mk.ru/politics/2021/05/28/mer-rigi...,18:17,326,"МИД РФ, Александр Лукашенко, Белоруссия, Латвия"
В Европе разрешили вакцинацию подростков от коронавируса,https://www.mk.ru/social/health/2021/05/28/v-e...,18:10,242,Ангела Меркель
...,...,...,...,...
Источник назвал местонахождение Протасевича,https://www.mk.ru/politics/2021/05/28/istochni...,10:02,5895,Белоруссия
"Лукашенко сообщил, что СНГ подтвердил свою жизнеспособность в сложное время",https://www.mk.ru/politics/2021/05/28/lukashen...,09:57,1481,"Александр Лукашенко, Москва, Белоруссия"
ЦБ отозвал лицензии у двух банков,https://www.mk.ru/economics/2021/05/28/cb-otoz...,09:48,2151,
Лукашенко заявил о прямой угрозе целостности Белоруссии,https://www.mk.ru/politics/2021/05/28/lukashen...,09:46,2307,"Александр Лукашенко, Белоруссия"
